In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import csv

In [6]:
#Main URL
url = 'https://www.flipkart.com/mens-footwear/sports-shoes/pr?sid=osp%2Ccil%2C1cu&otracker=nmenu_sub_Men_0_Sports+Shoes&page=1'

In [7]:
#function to find the html file
def responce(url):
    headers={"accept-language": "en-US,en;q=0.9","accept-encoding": "gzip, deflate, br","User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36","accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7"}
    responce = requests.get(url, headers = headers) 
    soup =BeautifulSoup(responce.text, 'lxml')
    return(soup)
    

In [23]:
#functions to find the various attributes of the product
def company_name(data):
    try:
     company = data.find('span', class_ = 'mEh187').get_text().split('\xa0')[0]
    except AttributeError:
     company = "Not Found"
    return(company)

def product_name(data):
    try:
     product = data.find('span', class_ = 'VU-ZEz').get_text().split('\xa0')[0]
    except AttributeError:
     product = "Not Found"
    return(product)

def final_price(data):
    try:
        price = data.find('div', class_ = 'Nx9bqj CxhGGd').get_text()[1:]
    except AttributeError:
        price = "Not Found"
    return(price)

def color(data):
    try:
        color = data.find('div', class_ = 'col col-9-12 -gXFvC').get_text().strip()
    except AttributeError:
        color = "Not Found"
    return(color)
    
def outer_material(data):
    try:
        for i in data.find_all('div', class_ = 'col col-9-12 -gXFvC')[1]:
            material = i.get_text().strip()
    except AttributeError:
        material = "Not Found"
    except IndexError: 
        material = "Not Found"
    return(material)
    
def model(data):
    try:
        for i in data.find_all('div', class_ = 'col col-9-12 -gXFvC')[2]:
            m_odel = i.get_text().strip()
    except AttributeError:
        m_odel = "Not Found"
    except IndexError: 
        m_odel = "Not Found"
    return(m_odel)

def rating(data):
    try:
        rating = data.find('div', class_ = 'XQDdHH _1Quie7').get_text()
    except AttributeError:
        rating = 'Rating Not Available'
    return(rating)
        
def no_review(data):
    try: 
        review_no = data.find('span', class_ = 'Wphh3N').get_text().split('reviews')[0].split('and')[1]
    except AttributeError:
        review_no = 'No rating Found'
    return(review_no)
 
def no_rating(data):
    try: 
        rating_no = data.find('span', class_ = 'Wphh3N').get_text().split('ratings')[0]
    except AttributeError:
        rating_no = 'No rating Found'
    return(rating_no)

In [24]:
#function to find the link for the next page of the link
def next_page(soup):
  try: 
    links = soup.find_all('a', class_ = '_9QVEpD')
    if len(links) != 1:
     product_url = 'https://www.flipkart.com'+ soup.find_all('a', class_ = '_9QVEpD')[1]['href']
    else:
     product_url = 'https://www.flipkart.com'+ soup.find('a', class_ = '_9QVEpD')['href']
  except TypeError:
    product_url = None
  except IndexError:
    product_url = None
  return(product_url)

In [27]:
#main function which will give the desired output
def main(url):
    main_data = []#it is the final data file
    while True:
        data = []#it is the temp data file to hold the data during each etration
        soup = responce(url)
        links = soup.find_all('a', class_ = '+tlBoD')
        product_urls = []
        for l in links:#the loop to find the links for each of the products
                list = 'https://www.flipkart.com'+ str(l.get('href'))
                product_urls.append(list)
        for p in product_urls:#loop to find the attribute of each of the product
                soup2 = responce(p)
                info = {"company" : company_name(soup2),
                        "product" : product_name(soup2),
                        "color": color(soup2),
                        "price" : final_price(soup2),
                        "material": outer_material(soup2),
                        "model" : model(soup2),
                        "Avg_Rating": rating(soup2),
                        "No. Reviews": no_review(soup2),
                        "No. Rating": no_rating(soup2),
                        "link": p}
                data.append(info)
        main_data.extend(data)
        url = next_page(soup)
        if url == None:
            break
    return(main_data)


In [28]:
#create the data set of the final data
shoe_dataSet = pd.DataFrame(main(url))
print(shoe_dataSet)

       company                                            product  \
0     RED TAPE  Athleisure Sports Shoes for Men | Enhanced Dur...   
1     RED TAPE  Atheleisure Sports Shoes for Men | Soft Cushio...   
2       REEBOK                  SUPER SPEED Walking Shoes For Men   
3       ADIDAS                        CriNu Cricket Shoes For Men   
4       ADIDAS        X CRAZYFAST CLUB FxG Football Shoes For Men   
..         ...                                                ...   
995  Not Found                                          Not Found   
996     REEBOK                         Run Snazzy Casuals For Men   
997     REEBOK              Emergo runner M Running Shoes For Men   
998       PUMA            PUMA Turf Slip On Running Shoes For Men   
999      asian  Carbon-02 Gray Sports,Gym,Training,Walking,Sty...   

            color      price  \
0     Blue, Green      1,539   
1           White      1,379   
2           Black      1,047   
3           White      1,397   
4          

In [1]:
#exporting the database to csv file
shoe_dataSet.to_csv('C:/Users/a s bhati/Downloads/Sports Shoes for Men.csv', index = False)

NameError: name 'shoe_dataSet' is not defined